**Импорт библиотек для работы**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
#Загрузка датасета

realty_data = pd.read_csv("C:\\Users\\79960\\OneDrive\\Документы\\Итоговый проект\\data.csv")

In [ ]:
realty_data.sample(3)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
187763,for sale,NaN,lot/land,5640 NW 6th Ave,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Miami,"[{'rating': ['2/10', '2/10', '10/10', '3/10', ...","3,509 sqft",33127,NaN,FL,NaN,NaN,NaN,A10783162,"$70,000"
173918,Under Contract Show,NaN,Single Family Home,6327 Big Oak Ln,2,"{'atAGlanceFacts': [{'factValue': '1979', 'fac...",Den,Charlotte,"[{'rating': ['2', '2', '3', '6', '3', '6', 'NR...",1505,28227,3,NC,1 Story,NaN,NaN,3512475,"$149,000"
44562,Active,NaN,Contemporary,852 Park Grove Ct,"1,750","{'atAGlanceFacts': [{'factValue': '2005', 'fac...",NaN,ORLANDO,"[{'rating': ['9', '7', '6', '8', '7', '7'], 'd...","1,312",32828,Baths,FL,Two,NaN,NaN,O5831859,"205,000"


In [ ]:
realty_data.shape

(377185, 18)

# Предобработка данных

**Работа с множественными пропущенными значениями.**

In [ ]:
num_rows = len(realty_data.index)
low_information_cols = []

for col in realty_data.columns:
    cnts = realty_data[col].value_counts(dropna=False)
    top_pct = (cnts/num_rows).iloc[0]

    if top_pct > 0.50:           #количество пропущенных значений более 50%
        low_information_cols.append(col)
        print('{0}: {1:.5f}%'.format(col, top_pct*100))
        print(cnts)
        print()

private pool: 98.89153%
private pool
NaN    373004
Yes      4181
Name: count, dtype: int64

fireplace: 72.66222%
fireplace
NaN                                                                     274071
yes                                                                      50356
Yes                                                                      20856
1                                                                        14544
2                                                                         2432
                                                                         ...  
Free-standing, Insert, Wood                                                  1
Wood Burning, Attached Fireplace Doors/Screen, Electric, Gas Starter         1
One, Living Room                                                             1
FAMILYRM, Great Room, Living Room                                            1
Ceiling Fan, SMAPL, Utility Connection, Walk-In Closets                      1
Name: co

Как мы видим из вычислений большое количество пропущенных значений имеют колонки: private pool, PrivatePool,fireplace,mls-id.

А колонки **private pool**, **PrivatePool** дают нам одинаковое представление(дублирующие колонки).
Данный признак может по разному влиять на цену недвижимости в разных  климатических зонах(в северной части- скорее уменьшить,в южной части-скорее увеличить),поэтому нужно оставить этот признак и посмотреть,как он может влиять на данные.


Ссылка на статью:https://www.nytimes.com/2021/07/17/realestate/pool-property-value.html



Также мы видим большое количество пропущенных значений в колонке **mls-id**. Для этой колонки также существует дублирующая MlsId.
MLS ID – его можно назвать общедоступным идентификатором или идентификатором агента.
Поэтому от данных колонок нужно отказаться.

Ссылка на статью:https://www.sapling.com/5834082/listing-mls-id-number




In [ ]:
realty_data = realty_data.drop('MlsId', axis=1)
realty_data = realty_data.drop('mls-id', axis=1)

In [ ]:
realty_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103114 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226469 non-null  object
 14  PrivatePool   40311 non-null   object
 15  target        374704 non-null  object
dtypes: object(16)
memory usage: 46.0+ MB


**Удаление дубликатов**

In [ ]:
realty_data = realty_data.drop_duplicates(ignore_index=True)

**Status** *статус объекта недвижимости*

In [ ]:
missing_values = realty_data.status.isna().sum()
total_count = len(realty_data)

print(f"Количество пропущенных значений: {missing_values}\n"
      f"Количество уникальных значений: {realty_data.status.nunique()}\n"
      f"Уникальные значения: {list(realty_data.status.unique()[:20])}")

Количество пропущенных значений: 39916
Количество уникальных значений: 159
Уникальные значения: ['Active', 'for sale', nan, 'New construction', 'New', 'For sale', 'Pending', 'P', 'Active/Contingent', 'Pre-foreclosure / auction', ' / auction', 'Under Contract', 'Under Contract   Showing', 'Pre-foreclosure', 'Under Contract Backups', 'foreclosure', 'Active Under Contract', 'Foreclosed', 'Option Pending', 'Under Contract Show']


In [ ]:
#Категоризация статусов для уменьшения их количества

categories = {
    "Coming Soon": [
        'Coming soon: Dec 23.','Coming soon: Nov 25.','Coming soon: Nov 12.','Coming soon: Nov 5.','Coming soon: Dec 1.',
        'Coming soon: Dec 11.', 'Coming soon: Dec 5.', 'Coming soon: Dec 7.', 'Coming soon: Dec 27.', 'Coming soon: Nov 14.',
        'Coming soon: Nov 22.', 'Coming soon: Oct 21.','Coming soon: Nov 13.','Coming soon: Dec 3.','Coming soon: Dec 25.',
        'Coming soon: Nov 11.', 'Coming soon: Dec 14.', 'Coming soon: Oct 24.','Coming soon: Nov 28.','Coming soon: Nov 19.',
        'Coming soon: Nov 8.', 'Coming soon: Oct 29.','Coming soon: Dec 15.', 'Coming soon: Dec 20.', 'Coming soon: Dec 9.',
        'Coming soon: Nov 27.','Coming soon: Nov 26.','Coming soon: Dec 6.','Coming soon: Dec 24.','Coming soon: Dec 10.',
        'Coming soon: Nov 17.', 'Coming soon: Dec 13.','Coming soon: Oct 30.','Coming soon: Nov 30.','Coming soon: Nov 21.',
        'Coming soon: Dec 4.','Coming soon: Dec 18.','Coming soon: Nov 29.', 'Coming soon: Dec 2.','Coming soon: Dec 16.',
         'Coming soon: Dec 12.','Coming soon: Nov 23.'],

    "Active": [
        'Active','For Sale', 'New construction', 'New', 'For sale','Lease/Purchase','A Active','Uc Continue To Show',
        'Price Change','Reactivated','Back On Market','Re Activated','Reactivated','Back on Market','Active with Contract',
        'Temporary Active','Active Option', 'Active Option Contract','Active With Offer','Active With Contingencies'],

    'Sold': [
           'Sold: $285,000','Sold: $15,000','Closed','recently sold','Accepted Offer'],

    'Foreclosed': [
        'Pre-foreclosure / auction','Pre-foreclosure','Closed','recently sold','Pf','Pi','Ps','Foreclosed','foreclosure',
        'Foreclosure','Contingent   Foreclosure','Pending Backups Wanted', 'Pending Fe'],

    'Auction': ['/ auction','Auction','Auction - Active','Active - Auction'],

    'Pending':[
        'Pending','P','Option Pending','Due Diligence Period','Pending Sh','Pending Offer Approval',
        'Pending In', 'Pending W/Insp Finance', 'Pending   Continue To Show   Financing',
        'Pending W/ Cont.', 'Pending W/Escape Clause','Pending With Contingencies',
        'Lease/Purchase Pending', 'Pending Bring Backup', 'Pending Taking Backups','P Pending Sale','Pending Sale',
        'Pending Sale','Pending, Continue to Show','Pending - Backup Offer Requested','Pending W/Backup Wanted',
        'Pending   Continue To Show','Pending (Do Not Show)', 'Pending - Taking Backups','Pending - Continue to Show',
        'Pending Continue To Show','Pending Ab','pending', 'Pending Inspection','Pending Take Backups'],

    'For rent':['Apartment for rent','Condo for rent','for rent'],

    'Backup Contract':['Contract','Active Under Contract','Under Contract','U Under Contract','Contract Contingent On Buyer Sale',
                       'Under Contract W/ Bckp','Under Contract Taking Back Up Offers','Backup','Contract P', 'Under Contract   Showing',
                       'Under Contract Backups','Under Contract Show','Under Contract - No Show', 'Active Backup',
                       'Under contract','Contingent Take Backup','Backup Contract', 'Option Contract','Conditional Contract',
                       'Under Contract - Show','Conting Accpt Backups','Accepting backups'],

    'Contingent':['CT Insp - Inspection Contingency','C Continue Show','Ct','Contingent','Contingency Contract',
                  'Active Contingency','Active/Contingent','Contingent   No Show','Contingent - Sale of Home',
                  'Contingent - Financing','Contingency 48 Hr (+/ )','Contingent   Release','Contingent Lien Holder Release',
                  'Contingent Show','Contingent Escape','Active Contingent','Contingent   Show','C', 'Contingent Finance And Inspection',
                  'Contingent Finance and Inspection','Active - Contingent'],

     'Listing Extended': ['Listing Extended']
}

def separation_statuses(status_value):
    if isinstance(status_value, str): # Проверяем, является ли значение строкой
        for category, values in categories.items():
            if status_value.lower() in [val.lower() for val in values]:
                return category
    return "other"


# Заменяем значения столбца 'status' на категории, используя функцию separation_statuses
realty_data['status'] = realty_data['status'].apply(separation_statuses)

In [ ]:
#Проверка полученных категорий
list(realty_data.status.unique())

['Active',
 'other',
 'Pending',
 'Contingent',
 'Foreclosed',
 'Backup Contract',
 'For rent',
 'Auction',
 'Sold',
 'Coming Soon',
 'Listing Extended']

**propertyType** *тип недвижимости*

In [ ]:
missing_values_t = realty_data.propertyType.isna().sum()


print(f"Количество пропущенных значений: {missing_values_t}\n"
      f"Количество уникальных значений: {realty_data.propertyType.nunique()}\n"
      f"Уникальные значения: {list(realty_data.propertyType.unique()[:20])}")

Количество пропущенных значений: 34725
Количество уникальных значений: 1280
Уникальные значения: ['Single Family Home', 'single-family home', 'lot/land', 'townhouse', 'Florida', nan, 'Single Family', 'coop', 'English', '2 Story', 'Townhouse', 'multi-family', 'Penthouse, Split-Level', 'Multi-Family Home', 'Condo', 'condo', 'Land', 'Condo/Townhome/Row Home/Co-Op', ' ', 'Detached, Two Story']


In [ ]:
# переведем все в строчные
realty_data.propertyType = realty_data.propertyType.str.lower()

In [ ]:
# приведем разное написание к одному виду,yes и unknown заменим  на Other
realty_data.propertyType = realty_data.propertyType \
.str.replace('single-family home','single family') \
.str.replace('single family home','single family') \
.str.replace('yes','Other') \
.str.replace('multi-family home','multi-family') \
.str.replace('unknown','Other')

# создаем признак Type и записываем в него только первое слово
realty_data['Type'] = realty_data['propertyType'].str.split(',').str[0]

In [ ]:
#Категоризация типов недвижимости для уменьшения их количества

keyword_mapping_type = {
    'single_family': [
        'single family', '1 story', '2 story', 'detached', '1 story/ranch', '1 story traditional', 'single detached','singlefamilyresidence',
        'single wide', 'single-wide mobile with land','tri-level','a-frame','2 stories with basement','cape',
        'two story', 'one story', 'one level unit', 'rancher', '1 1/2 story', 'single wide mh', 'one story traditional'],

    'multi_family': [
        'multi-family', 'duplex', 'triplex','4-plex', 'fourplex', 'multi_level','multi-level', 'multi family', '2 unit condo',
        'multiplex', '2-story'],

    'condos': [
        'condo', 'coop', 'cooperative', 'condo/townhome/row home/co-op', 'condo/townhome', 'condominium', 'condo/unit',
        'apartment/condo/townhouse', 'co-op', '2 story condo', 'high rise','condominium (3+ levels)','condominium (2 levels)'],

    'townhouse': [
        'townhouse', 'townhome style', 'townhouse-interior', 'townhouse-end unit'],

    'apartment': [
        'apartment', 'condominium (single level)', 'high-rise', 'mid-rise', 'low-rise (1-3 stories)', 'flat', 'studio'],

    'land': [
        'lot/land', 'land'],

    'mobile_home': [
        'mobile/manufactured', 'mobile / manufactured', 'manufactured house', 'mfd/mobile home', 'mobile home',
        'manufactured home', 'manufactured double-wide', 'manufactured single-wide', 'mobile home 1 story',
        'mobile manu - double wide','single-wide mobile with land','manuf/mobile'],

    'ranch': [
        'ranch', 'one story','step ranch','hi ranch','raised ranch'],

    'contemporary ': [
        'contemporary', 'contemporary/modern', "modern", 'mid-century modern', 'modern farmhouse', 'modernist','urban contemporary'],

    'historical': [
        'historical', 'designated historical home', 'historical/conservation district', 'historic/older', 'historic vintage',
        'historic'],

    'other': [
        'Other','other (see remarks)', 'english','spanish','french', 'other style','victorian ', 'florida', 'farms/ranches', 'carriage house', 'country english', 'straight thru', 'less than 4 floors', 'bungalow', 'custom', 'arts & crafts', 'tudor', 'new build 2019', 'split foyer', 'cottage', 'cottage/camp', 'garden home', 'farm/ranch', 'farm/ranch house', 'farm house', 'attached duplex', 'farmhouse', 'houseboat', 'ground floor', '3 story', '3+ stories', 'santa barbara/tuscan', 'old style', 'modular/prefab', 'post and beam', 'multiple occupancy', 'attached', 'hawaiian plantation', 'forest garden home', '1 1/2 story with basement', 'split-entry', 'texas hill country', 'lake house', '1 story with basement', 'hi-rise', 'coastal beach home', 'historical', 'key west/coastal', 'loft/balcony', 'english manor', 'mid-rise (4-7 stories)', 'mid-level', 'new englander', 'residential (<1 acre)', 'ranch','residential (1+ acre)', 'split', 'split level', 'split (4 level)', 'split (5+ level)', 'urban', 'patio', 'patio home', 'penthouse', 'manor', 'victorian/federal', 'coastal', 'coastal contemporary', 'coastal ii', 'coastal modern', 'coastal two story','elevated ', 'mountain contemporary', 'key west', 'high ranch', 'end unit']
}

#функция,которая принимает значение и возращает тип недвижимости,если значение не найдено,возвращает other
def separation_propertyType(value):
    for key, type_values in keyword_mapping_type.items():
        if value in type_values:
            return key
    return "other"


realty_data['Type'] = realty_data['Type'].apply(separation_propertyType)

In [ ]:
realty_data.drop('propertyType', axis=1, inplace=True)

**street** *улица*

In [ ]:
missing_values_s = realty_data.street.isna().sum()


print(f"Количество пропущенных значений: {missing_values_s}\n"
      f"Количество уникальных значений: {realty_data.street.nunique()}\n"
      f"Уникальные значения: {list(realty_data.street.unique()[:20])}")

Количество пропущенных значений: 2
Количество уникальных значений: 337076
Уникальные значения: ['240 Heather Ln', '12911 E Heroy Ave', '2005 Westridge Rd', '4311 Livingston Ave', '1524 Kiscoe St', '1624 S Newkirk St', '552 Casanova Ct ', '6094 Mingle Dr ', '11182 Owl Ave', '8612 Cedar Plains Ln', '14205 Roosevelt Ave #512', '4222 False Cypress Ln ', '16373 NW 16th St', '1538 Umpqua Dr', '15351 Falmouth Ave', '930 52nd St ', '2402 NW 22nd Rd ', '24 W 126th St ', '22702 Alderdale Ln', '1633 NE 15th St']


In [ ]:
# удаляем строки, где есть пропуски в столбце street
realty_data = realty_data.dropna(subset=['street'])

**PrivatePool** *приватный бассейн*

In [ ]:
# Объединяем колонки private pool, PrivatePool
realty_data['pool'] = realty_data['private pool'].combine_first(realty_data['PrivatePool'])

# Заменить NaN на False
realty_data['pool'] = realty_data['pool'].fillna(False)

# Заменить 'yes' на True
realty_data['pool'] = realty_data['pool'].replace(['yes', 'Yes'], True)

# Выводим результат
print("Уникальные значения :", realty_data['pool'].unique())

# Удаляем столбец PrivatePool
realty_data = realty_data.drop(['PrivatePool', 'private pool'], axis=1)

Уникальные значения : [False  True]


In [ ]:
realty_data = realty_data.drop_duplicates(ignore_index=True)

**baths** *ванные комнаты*

In [ ]:
missing_values_b = realty_data.baths.isna().sum()


print(f"Количество пропущенных значений: {missing_values_b}\n"
      f"Количество уникальных значений: {realty_data.baths.nunique()}\n"
      f"Уникальные значения: {list(realty_data.baths.unique()[:20])}")

Количество пропущенных значений: 106260
Количество уникальных значений: 229
Уникальные значения: ['3.5', '3 Baths', '2 Baths', '8 Baths', nan, '2', '3', 'Bathrooms: 2', '1,750', '4 Baths', '2 ba', 'Bathrooms: 5', '1,000', '7 Baths', '2.0', '3.0', 'Bathrooms: 1', '4.0', '2.1 Baths', '2.5 Baths']


In [ ]:
mode_type_baths = realty_data['baths'].mode()[0]  # заменяем пропущенные значение модой
realty_data['baths'].fillna(mode_type_baths, inplace=True)

In [ ]:
realty_data.baths.unique()

array(['3.5', '3 Baths', '2 Baths', '8 Baths', '2', '3', 'Bathrooms: 2',
       '1,750', '4 Baths', '2 ba', 'Bathrooms: 5', '1,000', '7 Baths',
       '2.0', '3.0', 'Bathrooms: 1', '4.0', '2.1 Baths', '2.5 Baths', '1',
       'Bathrooms: 3', '4.5', '6 Baths', 'Bathrooms: 4', '3 ba', '5',
       '2,500', '5.5 Baths', '1.0', '5 Baths', '1.5', '4', '~', '2.5',
       '4,000', '3.5 Baths', '2,000', '3,000', '8.0', '1 ba', '3.5 ba',
       '0', '5.0', '1,500', '7.0', '1,250', '9 Baths', '2,250', '6.0',
       '12 Baths', '5.5', '3,500', '1.5 Baths', '2,750', 'Bathrooms: 6',
       '4.5 Baths', '750', '5.5+', '6', '10 Baths', '6 ba',
       'Bathrooms: 19', '10.0', '1.5 ba', '4 ba', '12 ba', '2.5+', '8',
       '7.5+', 'Bathrooms: 10', '0 / 0', 'Sq. Ft. ', '5 ba', '4.5+',
       '18 Baths', '-- baths', 'Bathrooms: 7', '7', '18', '3.5+', '1.5+',
       '11 Baths', '5,000', '1.75 Baths', '9', '12.0', '1.1 Baths', '6.5',
       'Bathrooms: 8', '10', '19 Baths', 'Bathrooms: 9', '16 Baths',
     

In [ ]:
# отдельно исправим некорректно написанные значения

realty_data['baths'].replace(['1,750'], 1.75, inplace=True)
realty_data['baths'].replace(['1,000'], 1, inplace=True)
realty_data['baths'].replace(['2,500'], 2.5, inplace=True)
realty_data['baths'].replace(['~','0 / 0','Sq. Ft. ','-- baths','—'],0, inplace=True)
realty_data['baths'].replace(['4,000'], 4, inplace=True)
realty_data['baths'].replace(['3,000'], 3, inplace=True)
realty_data['baths'].replace(['1,500'], 1.5, inplace=True)
realty_data['baths'].replace(['1,250'], 1.25, inplace=True)
realty_data['baths'].replace(['2,250'], 2.25, inplace=True)
realty_data['baths'].replace(['3,500'], 3.5, inplace=True)
realty_data['baths'].replace(['2,750'], 2.75, inplace=True)
realty_data['baths'].replace(['5,000'], 5, inplace=True)
realty_data['baths'].replace(['7,500+'], 7.5, inplace=True)
realty_data['baths'].replace(['1-0 / 1-0 / 1','1 / 1 / 1 / 1','1 / 1-0 / 1-0 / 1-0'], 1, inplace=True)
realty_data['baths'].replace(['116 / 116 / 116'], 116, inplace=True)
realty_data['baths'].replace(['3-1 / 2-2'], 3, inplace=True)
realty_data['baths'].replace(['2-1 / 2-1 / 1-1 / 1-1'], 2, inplace=True)
realty_data['baths'].replace(['1-2 Baths'], 2, inplace=True)

In [ ]:
realty_data['baths'] = realty_data['baths'].apply(lambda x: float(re.findall('\d+', str(x))[0]) if re.findall('\d+', str(x)) else None)

In [ ]:
realty_data.query('baths > 200')

,status,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,Type,pool
1109,Active,3624 Douglas Ave,750.0,"{'atAGlanceFacts': [{'factValue': '1993', 'fac...",1,Dallas,"[{'rating': ['5', '5', '2', '10'], 'data': {'D...","1,758",75219,Baths,TX,NaN,"499,000",other,False
2022,Active,920 I St NW #411,750.0,"{'atAGlanceFacts': [{'factValue': '2013', 'fac...",NaN,Washington,"[{'rating': ['8', '8', '3', '8'], 'data': {'Di...","1,367",20001,NaN,DC,NaN,"1,499,000",other,False
3594,Active,601 W 11th Ave #409,750.0,"{'atAGlanceFacts': [{'factValue': '1981', 'fac...",NaN,DENVER,"[{'rating': ['5', 'NR', '5', '2'], 'data': {'D...",873,80204,Baths,CO,NaN,"348,900",other,False
7564,Active,1260 Xavier St,750.0,"{'atAGlanceFacts': [{'factValue': '1936', 'fac...",NaN,DENVER,"[{'rating': ['2', '2', '2'], 'data': {'Distanc...","1,864",80204,Baths,CO,NaN,"560,000",other,False
7693,Active,3314 Douglas Ave Unit 202D,750.0,"{'atAGlanceFacts': [{'factValue': '1984', 'fac...",3,Dallas,"[{'rating': ['4', '3', '2', '10'], 'data': {'D...","1,336",75219,Baths,TX,3,"285,000",condos,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372463,Active,601 Pennsylvania Ave NW #212,750.0,"{'atAGlanceFacts': [{'factValue': '1991', 'fac...",NaN,Washington,"[{'rating': ['8', '8', '3', '5'], 'data': {'Di...",801,20004,NaN,DC,NaN,"449,000",other,False
375492,Active,3225 Turtle Creek Blvd #1646,750.0,"{'atAGlanceFacts': [{'factValue': '1998', 'fac...",NaN,Dallas,"[{'rating': ['5', '5', '2', '10'], 'data': {'D...",832,75219,Baths,TX,NaN,"219,500",condos,False
375870,Active,2262 Alterbrook Ln,750.0,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Dallas,"[{'rating': ['4', '3', '2', '10'], 'data': {'D...","2,102",75219,Baths,TX,NaN,"464,900",condos,False
376005,Active,3225 Turtle Creek Blvd #528,750.0,"{'atAGlanceFacts': [{'factValue': '1998', 'fac...",NaN,Dallas,"[{'rating': ['5', '5', '2', '10'], 'data': {'D...",837,75219,Baths,TX,22,"240,000",condos,False


Вывод из полученных данных: существует 238 объектов недвижимости,которые имеют более 200 ванных комнат,однако даже в самых огромных объектах количество спален не более 200.

In [ ]:
# не включаем данные строки
realty_data = realty_data.loc[realty_data['baths'] <= 100]

**homeFacts** *данные о здании*

In [ ]:
realty_data['homeFacts'][0]

"{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Central A/C, Heat Pump', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': '$144', 'factLabel': 'Price/sqft'}]}"

Достаем label и Value из признака homeFacts

In [ ]:
label = realty_data.homeFacts.str.findall(r"\bfactLabel': ([\s\S]+?)[}\b]")
print(label[0])

Value = realty_data.homeFacts.str.findall(r"\bfactValue': ([\s\S]+?), 'factLabel\b")
print(Value[:3])

["'Year built'", "'Remodeled year'", "'Heating'", "'Cooling'", "'Parking'", "'lotsize'", "'Price/sqft'"]
0    ['2019', '', 'Central A/C, Heat Pump', '', '',...
1    ['2019', '', '', '', '', '5828 sqft', '$159/sq...
2    ['1961', '1967', 'Forced Air', 'Central', 'Att...
Name: homeFacts, dtype: object


In [ ]:
# список названий признаков без кавычкек
list_label = ','.join(label[0]).replace("'","").split(',')

# Создаем признаки в датафрейме и заполняем  значениями
for i, value in enumerate(list_label):
    realty_data[value]=Value.apply(lambda x: x[i])


In [ ]:
realty_data = realty_data.drop('homeFacts', axis=1)

**Year built** *год постройки*

In [ ]:
#удаляем кавычки вокруг чисел
realty_data['Year built'] = realty_data['Year built'].str.replace("'",'', regex=True)

In [ ]:
realty_data['Year built'].sort_values().unique()

array(['', '1', '1019', '1057', '1060', '1208', '1700', '1703', '1735',
       '1740', '1750', '1780', '1788', '1790', '1794', '1795', '1796',
       '1799', '1800', '1802', '1803', '1804', '1805', '1807', '1808',
       '1809', '1810', '1811', '1812', '1815', '1816', '1817', '1818',
       '1820', '1822', '1823', '1824', '1825', '1828', '1829', '1830',
       '1831', '1832', '1834', '1835', '1836', '1840', '1842', '1843',
       '1844', '1845', '1846', '1847', '1848', '1850', '1851', '1852',
       '1853', '1854', '1855', '1856', '1857', '1858', '1859', '1860',
       '1861', '1862', '1863', '1864', '1865', '1866', '1867', '1868',
       '1869', '1870', '1871', '1872', '1873', '1874', '1875', '1876',
       '1877', '1878', '1879', '1880', '1881', '1882', '1883', '1884',
       '1885', '1886', '1887', '1888', '1889', '1890', '1891', '1892',
       '1893', '1894', '1895', '1896', '1897', '1898', '1899', '1900',
       '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908',
     

In [ ]:
# Заменим кавычки на no date
realty_data['Year built'] = realty_data['Year built'].str.replace('^\s*$','no date', regex=True)
# Заменим некорректные значения на no date
mask= realty_data['Year built'].isin(['','1','1208','559990649990','2025','No Data','None'])
realty_data.loc[mask,'Year built'] = 'no date'


# Исправим некорректно написанные значения
realty_data.loc[realty_data['Year built']=='1019','Year built'] = '1919'
realty_data.loc[realty_data['Year built']=='1057','Year built'] = '1957'
realty_data.loc[realty_data['Year built']=='1060','Year built'] = '1960'

In [ ]:
realty_data['Year built'].sort_values().unique()

array(['1700', '1703', '1735', '1740', '1750', '1780', '1788', '1790',
       '1794', '1795', '1796', '1799', '1800', '1802', '1803', '1804',
       '1805', '1807', '1808', '1809', '1810', '1811', '1812', '1815',
       '1816', '1817', '1818', '1820', '1822', '1823', '1824', '1825',
       '1828', '1829', '1830', '1831', '1832', '1834', '1835', '1836',
       '1840', '1842', '1843', '1844', '1845', '1846', '1847', '1848',
       '1850', '1851', '1852', '1853', '1854', '1855', '1856', '1857',
       '1858', '1859', '1860', '1861', '1862', '1863', '1864', '1865',
       '1866', '1867', '1868', '1869', '1870', '1871', '1872', '1873',
       '1874', '1875', '1876', '1877', '1878', '1879', '1880', '1881',
       '1882', '1883', '1884', '1885', '1886', '1887', '1888', '1889',
       '1890', '1891', '1892', '1893', '1894', '1895', '1896', '1897',
       '1898', '1899', '1900', '1901', '1902', '1903', '1904', '1905',
       '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913',
      

**Remodeled year** *год реконструкции*

In [ ]:
#удаляем кавычки вокруг чисел
realty_data['Remodeled year'] = realty_data['Remodeled year'].str.replace("'",'', regex=True)

In [ ]:
realty_data['Remodeled year'].sort_values().unique()

array(['', '0', '1111', '1738', '1800', '1845', '1846', '1853', '1862',
       '1869', '1870', '1874', '1876', '1877', '1880', '1883', '1884',
       '1885', '1886', '1887', '1888', '1889', '1890', '1891', '1892',
       '1893', '1894', '1895', '1896', '1897', '1898', '1899', '1900',
       '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908',
       '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916',
       '1917', '1918', '1919', '1920', '1921', '1922', '1923', '1924',
       '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932',
       '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940',
       '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948',
       '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956',
       '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972',
       '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
     

In [ ]:
#Заменяем некоректные значения на None

mask_y= realty_data['Remodeled year'].isin(['', '0', '1111'])
realty_data.loc[mask_y,'Remodeled year'] = 'None'

In [ ]:
# посмотрим на колличество None
print(f"Процент None: {round(realty_data[realty_data['Remodeled year']=='None'].shape[0]/realty_data.shape[0]*100, 2)}%")
print(f"Количество None: {realty_data.loc[realty_data['Remodeled year']=='None', 'Remodeled year'].count()}")

Процент None: 60.02%
Количество None: 226144


Колонка 'Remodeled year' содержит только около 140 000  из 377000 строк не имеющих значение  Nope это около 40%.Удаляем данный признак.

In [ ]:
realty_data = realty_data.drop('Remodeled year', axis=1)

**Heating**  *отопление*

In [ ]:
realty_data['Heating'].str.lower().value_counts()

Heating
'forced air'                                                      134306
''                                                                105490
'other'                                                            29622
'electric'                                                         10216
'gas'                                                               9295
                                                                   ...  
'mini/split ac unit-master'                                            1
'zoned heating, wall unit heating, forced air heating'                 1
'baseboard, spacewallunit'                                             1
'hot air, stove-pellet'                                                1
'baseboard, hot water, programmable thermostat, radiant floor'         1
Name: count, Length: 1912, dtype: int64

Признак имеет  много пропусков и с большой вероятностью пропуски это отсутствие отпления

In [ ]:
# удаляем лишние кавычки
realty_data['Heating'] = realty_data['Heating'].str.replace("'",'', regex=True)
realty_data['Heating'] = realty_data['Heating'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none'] else False)

In [ ]:
# выполним проверку
missing_values_Heating = realty_data['Heating'].isna().sum()

print(f"Количество пропущенных значений: {missing_values_Heating}\n"
f"Количество уникальных значений: {realty_data['Heating'].nunique()}\n"
f"Уникальные значения: {list(realty_data['Heating'].sort_values().unique())}")

Количество пропущенных значений: 0
Количество уникальных значений: 2
Уникальные значения: [False, True]


**Cooling** *система охлаждения*

In [ ]:
realty_data['Cooling'].str.lower().value_counts()

Cooling
'central'                                             158741
''                                                    120272
'central air'                                          14345
'no data'                                              10615
'has cooling'                                           9730
                                                       ...  
'zoned cooling, ceiling fan(s)'                            1
'central, multiunits'                                      1
'central, electric, fans - attic'                          1
'electric heating, individual heating'                     1
'central a/c (gas), central heat (gas), heat pump'         1
Name: count, Length: 1436, dtype: int64

In [ ]:
realty_data['Cooling'] = realty_data['Cooling'].str.replace("'",'', regex=True)
realty_data['Cooling'] = realty_data['Cooling'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none'] else False)

Признак имеет  много пропусков и с большой вероятностью пропуски это отсутствие системы охлаждения

**Parking** *парковка*

In [ ]:
realty_data['Parking'].str.lower().value_counts()

Parking
''                                                                                                                                    171517
'attached garage'                                                                                                                      70748
'2 spaces'                                                                                                                             28060
'1 space'                                                                                                                              14252
'no data'                                                                                                                              13332
                                                                                                                                       ...  
'boat, driveway, garage door opener, garage faces rear, oversized, workshop in garage'                                                     1
'assi

Признак имеет  много пропусков и с большой вероятностью пропуски это отсутствие паркинга

In [ ]:
# удалим лишние кавычки
realty_data['Parking'] = realty_data['Parking'].str.replace("'",'', regex=True)
realty_data['Parking'] = realty_data['Parking'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none','0'] else False)

**lotsize** *размер участка*

In [ ]:
realty_data['lotsize'].str.lower().value_counts()

lotsize
''                  33055
none                28346
'—'                 25073
'no data'            5329
'-- sqft lot'        3819
                    ...  
'4,396 sqft'            1
'8,661 sqft'            1
'5,591 sqft'            1
'5,716 sq. ft.'         1
'7,084 sqft lot'        1
Name: count, Length: 36599, dtype: int64

In [ ]:
#Заменяем некоректные значения на None
realty_data['lotsize'] = realty_data['lotsize'].str.replace("'",'', regex=True)
mask_lot= realty_data['lotsize'].isin(['', 'none', '—','no data','-- sqft lot'])
realty_data.loc[mask_lot,'lotsize'] = 'None'


Удаляем данный признак так как он содержит много пропусков и некорректную информацию,которую невозможно перевести в значение lotsize

In [ ]:
realty_data.drop('lotsize', axis=1, inplace=True)

**Price/sqft** *цена/квадратный фут*

Признак Price/sqft напрямую зависит от целевого признака,поэтому удаляем.

In [ ]:
realty_data= realty_data.drop('Price/sqft', axis=1)

**fireplace** *камин*

In [ ]:
realty_data['fireplace'].str.lower().value_counts()

fireplace
yes                                                                     71209
1                                                                       14486
2                                                                        2418
not applicable                                                           1993
fireplace                                                                 847
                                                                        ...  
free-standing, insert, wood                                                 1
wood burning, attached fireplace doors/screen, electric, gas starter        1
one, living room                                                            1
familyrm, great room, living room                                           1
ceiling fan, smapl, utility connection, walk-in closets                     1
Name: count, Length: 1651, dtype: int64

Признак имеет много пропусков и с большой вероятностью пропуски это отсутствие камина

In [ ]:
realty_data['fireplace'] = realty_data['fireplace'].apply(lambda x: True if x not in ['', 'no data', 'None', 'none', '0', 'not applicable', 'no'] and not pd.isna(x) else False)

**City** *город*

In [ ]:
# Приведем названия к единому виду(с заглавной буквы)
realty_data['city'] = realty_data['city'].str.title()

missing_values_count = realty_data['city'].isna().sum()


print(f"Количество пропущенных значений: {missing_values_count}\n"
f"Количество уникальных значений: {realty_data['city'].nunique()}\n"
f"Уникальные значения: {list(realty_data['city'].sort_values().unique()[:10])}")

Количество пропущенных значений: 32
Количество уникальных значений: 1909
Уникальные значения: [' ', '--', 'Abilene', 'Abingdon', 'Accokeek', 'Adams', 'Addison', 'Adel', 'Adelphi', 'Advance']


In [ ]:
# Удаляем  слово Сity в названии
realty_data['city'] = realty_data['city'].str.replace(' City', '')

#удаляем строки, содержащие ' ', '--' или NaN в столбце 'city'
realty_data = realty_data[~realty_data['city'].isin([' ', '--', np.nan])]

**schools** *школы*

In [ ]:
realty_data.schools[3]

"[{'rating': ['9/10', '9/10', '10/10', '9/10'], 'data': {'Distance': ['1.05mi', '0.1mi', '1.05mi', '0.81mi'], 'Grades': ['5-6', 'PK-4', '7-8', '9-12']}, 'name': ['Mcculloch Intermediate School', 'Bradfield Elementary School', 'Highland Park Middle School', 'Highland Park High School']}]"

Данный признак содержит такие данные как:
                                         
                                          рейтинги школ,

                                          растояние до школы,
                                          
                                          классы обучения,
                                          
                                          название школы.

Классы обучения и название учебного заведения не дают полезной информации.А рейтинг школы и расстояние до школы могут стать нужными признаками(используются в американских агенствах по продаже недвижимости)

In [ ]:
# Средний рейтинг
def process_rating(rating):
    rating = rating[0].replace('/10', '')
    extracted_number = [float(num) for num in rating if num.isdigit()]
    average_rating = np.average(extracted_number) if extracted_number else -1
    return round(average_rating, 1)

a_rating = (realty_data.schools.str.findall(r"\brating': ([\s\S]+?), 'data\b").apply(lambda x: process_rating(x)))

# Создадим признак school_rating _mean в нашем наборе данных. Пропуски заполним -1
realty_data['school_rating _mean'] = a_rating

In [ ]:
# Расчет минимального расстояния до школы
s_distance = realty_data.schools.str.findall(r"\bDistance': ([\s\S]+?), 'Grades\b")
s_distance = s_distance.apply(lambda x: x[0])
s_distance = s_distance.str.replace('[a-zA-Z]','', regex=True)
# Удаление всех символов алфавита из расстояний
s_distance = s_distance.str.findall(r'\b([0-9]+.[0-9]+)')
s_distance = s_distance.apply(lambda x: [float(i) for i in x])

# Нахождение минимального значения в каждом списке s_distance, и возвращение -1, если длина списка равна 0
school_dist_min = s_distance.apply(lambda x: -1 if len(x)==0 else min(x))


realty_data['school_dist_min'] = school_dist_min

In [ ]:
realty_data.drop('schools', axis=1, inplace=True)

**sqft**  *Площадь в футах *

In [ ]:
missing_values_sqft = realty_data['sqft'].isna().sum()

print(f"Количество пропущенных значений: {missing_values_sqft}\n"
f"Количество уникальных значений: {realty_data['sqft'].nunique()}\n"
f"Уникальные значения: {list(realty_data['sqft'].sort_values().unique()[:20])}")

Количество пропущенных значений: 40509
Количество уникальных значений: 21097
Уникальные значения: ['--', '-- sqft', '0', '1', '1 sqft', '10 sqft', '100', '100 sqft', '1000', '1000 sqft', '10000', '10000 sqft', '100000 sqft', '10001', '100056 sqft', '10008 sqft', '10009', '10009 sqft', '1001', '1001 sqft']


In [ ]:
realty_data['sqft'] = realty_data['sqft'].str.replace(',', '')

In [ ]:
realty_data1 = realty_data.copy()

In [ ]:
def extract_number(x):
    # Преобразуем строку x в тип str, если она не already str
    x = str(x) if not isinstance(x, str) else x
    # Используем re.findall для поиска всех целых чисел в строке x
    numbers = re.findall('\d+', x)
    # Возвращаем первое найденное число, преобразованное в int, или None, если чисел нет
    return int(numbers[0]) if numbers else None

# Применяем функцию  и обновляем значения
realty_data1.loc[:, 'sqft'] = realty_data1['sqft'].apply(extract_number)

In [ ]:
realty_data1['sqft'] = realty_data1['sqft'].fillna(0)

In [ ]:
missing_values_sqft = realty_data1['sqft'].isna().sum()

print(f"Количество пропущенных значений: {missing_values_sqft}\n"
f"Количество уникальных значений: {realty_data1['sqft'].nunique()}\n"
f"Уникальные значения: {list(realty_data1['sqft'].sort_values().unique()[:20])}")

Количество пропущенных значений: 0
Количество уникальных значений: 9888
Уникальные значения: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 10.0, 11.0, 12.0, 17.0, 20.0, 22.0, 30.0, 31.0, 33.0, 35.0, 40.0, 41.0, 45.0]


**zipcode** *почтовый индекс *

In [ ]:
missing_values_zipcode = realty_data1['zipcode'].isna().sum()

print(f"Количество пропущенных значений: {missing_values_zipcode}\n"
f"Количество уникальных значений: {realty_data1['zipcode'].nunique()}\n"
f"Уникальные значения: {list(realty_data1['zipcode'].sort_values().unique()[:200])}")

Количество пропущенных значений: 0
Количество уникальных значений: 4549
Уникальные значения: ['--', '0', '00000', '02108', '02109', '02110', '02111', '02113', '02114', '02115', '02116', '02118', '02119', '02120', '02121', '02122', '02124', '02125', '02126', '02127', '02128', '02129', '02130', '02131', '02132', '02134', '02135', '02136', '02199', '02210', '02215', '02467', '04666', '04901', '04937', '04957', '04963', '05345', '05441', '05441-4400', '05443', '05444', '05445', '05448', '05450', '05454', '05455', '05456', '05457', '05461', '05462', '05464', '05469', '05472', '05473', '05478', '05482', '05483', '05487', '05488', '05491', '05642', '07004', '07004-1106', '07004-1300', '07004-1413', '07004-1528', '07004-1607', '07004-1809', '07004-1878', '07004-1935', '07004-2116', '07004-2137', '07004-2214', '08701', '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10009', '10010', '10011', '10012', '10013', '10014', '10016', '10017', '10018', '10019', '10021', '10022', '10023'

In [ ]:
realty_data1[(realty_data1['zipcode']=='--') | (realty_data1['zipcode']=='0')| (realty_data1['zipcode']=='00000') ]

,status,street,baths,fireplace,city,sqft,zipcode,beds,state,stories,target,Type,pool,Year built,Heating,Cooling,Parking,school_rating _mean,school_dist_min
30258,Active,Gates Canyon Rd,2.0,False,Vacaville,0.0,0,NaN,CA,NaN,"$380,000",land,False,no date,False,False,False,3.7,6.50
83514,Active,Cornejo Ricardo Descalzi,2.0,False,Quito Ecuador,0.0,0,NaN,NY,NaN,"470,000",other,False,1995,False,True,False,-1.0,-1.00
231228,Active,0 N Gopher Canyon Rd,2.0,False,Bonsall,0.0,--,449 acres,CA,NaN,"$60,000,000",other,False,no date,False,False,False,5.7,2.90
235152,other,1744 N Dixie Hwy # 1744,3.0,False,Fort Lauderdale,2043.0,--,3,FL,NaN,"$425,000",townhouse,False,2010,True,True,True,5.0,0.50
305479,Active,000 U.S. Hwy 359,2.0,False,Laredo,243849.0,0,NaN,TX,NaN,"$1,740,000",land,False,no date,False,False,False,4.0,7.45
308133,Active,435 pescador,2.0,False,Foreign Country,1100.0,00000,NaN,OS,NaN,"125,000",other,False,1972,False,False,False,-1.0,-1.00


In [ ]:
indexes_drop = realty_data1[(realty_data1['zipcode']=='--') | (realty_data1['zipcode']=='0')| (realty_data1['zipcode']=='00000') ].index

# Удаляем строки по  индексам
realty_data1 = realty_data1.drop(indexes_drop)

In [ ]:
#Исправим некорректные zipcode,которые имеют дополнительные числа после тире

realty_data1.zipcode = realty_data1.zipcode.str.replace("-.+",'', regex=True)

realty_data1['zipcode'].sort_values().unique()[:20]

array(['02108', '02109', '02110', '02111', '02113', '02114', '02115',
       '02116', '02118', '02119', '02120', '02121', '02122', '02124',
       '02125', '02126', '02127', '02128', '02129', '02130'], dtype=object)

**beds**  *количество спальных комнат*

In [ ]:
missing_values_beds = realty_data1['beds'].isna().sum()

print(f"Количество пропущенных значений: {missing_values_beds}\n"
f"Количество уникальных значений: {realty_data1['beds'].nunique()}\n"
f"Уникальные значения: {list(realty_data1['beds'].sort_values().unique()[:200])}")

Количество пропущенных значений: 91140
Количество уникальных значений: 1181
Уникальные значения: [' ', '# Bedrooms 1st Floor', '-- bd', '-- sqft', '0', '0.0', '0.25 acres', '0.26 acres', '0.27 acres', '0.28 acres', '0.29 acres', '0.3 acres', '0.31 acres', '0.32 acres', '0.33 acres', '0.34 acres', '0.35 acres', '0.36 acres', '0.37 acres', '0.38 acres', '0.39 acres', '0.4 acres', '0.41 acres', '0.42 acres', '0.43 acres', '0.44 acres', '0.45 acres', '0.46 acres', '0.47 acres', '0.48 acres', '0.49 acres', '0.5 acres', '0.51 acres', '0.52 acres', '0.53 acres', '0.54 acres', '0.55 acres', '0.56 acres', '0.57 acres', '0.58 acres', '0.59 acres', '0.6 acres', '0.61 acres', '0.62 acres', '0.63 acres', '0.64 acres', '0.65 acres', '0.66 acres', '0.67 acres', '0.68 acres', '0.69 acres', '0.7 acres', '0.71 acres', '0.72 acres', '0.73 acres', '0.74 acres', '0.75 acres', '0.76 acres', '0.77 acres', '0.78 acres', '0.79 acres', '0.8 acres', '0.81 acres', '0.82 acres', '0.83 acres', '0.84 acres', '0.85 a

К сожалению данный признак содержит во-первых много пропущенных значений,во-вторых текст описывающий недвижимость или ее площадь,в третьих слишком мало данных непосредственно о количестве спален.
Поэтому удаляем признак beds

In [ ]:
realty_data1 = realty_data1.drop('beds', axis=1)

**state** *штат*

In [ ]:
missing_values_state = realty_data1['state'].isna().sum()

print(f"Количество пропущенных значений: {missing_values_state}\n"
f"Количество уникальных значений: {realty_data1['state'].nunique()}\n"
f"Уникальные значения: {list(realty_data1['state'].sort_values().unique()[:20])}")

Количество пропущенных значений: 0
Количество уникальных значений: 37
Уникальные значения: ['AL', 'AZ', 'CA', 'CO', 'DC', 'DE', 'FL', 'Fl', 'GA', 'IA', 'IL', 'IN', 'KY', 'MA', 'MD', 'ME', 'MI', 'MO', 'MS', 'MT']


**stories** *этажи*

In [ ]:
missing_values_stories = realty_data1['stories'].isna().sum()

print(f"Количество пропущенных значений: {missing_values_stories}\n"
f"Количество уникальных значений: {realty_data1['stories'].nunique()}\n"
f"Уникальные значения: {list(realty_data1['stories'].sort_values().unique()[:400])}")

Количество пропущенных значений: 150321
Количество уникальных значений: 346
Уникальные значения: [', 1', ', 2', ', 3', '0', '0.0', '1', '1 1/2 Levels', '1 1/2 Story', '1 Level', '1 Level, 1.5 Level', '1 Level, 2 Level', '1 Level, Condo', '1 Level, Condo, Site Built', '1 Level, Non-Site Built', '1 Level, Site Built', '1 Level, Site Built, Townhouse', '1 Level, Split Foyer', '1 Level, Townhouse', '1 Leveland + Loft', '1 Story', '1 Story Basement', '1 Story, 2 Story', '1 Story, Hillside', '1 Story, Multi Level', '1 Story, Split Level', '1 Story, Townhouse', '1 Story/F.R.O.G.', '1 Story/Ranch', '1, 1', '1-2 Stories', '1-2 Stories, 3-4 Stories', '1.0', '1.00', '1.000', '1.0000', '1.2', '1.30', '1.3000', '1.5', '1.5 Level', '1.5 Level, Site Built', '1.5 Level, Site Built, Tri-Level', '1.5 Stories', '1.5 Story', '1.5 Story/Basement', '1.5, 2', '1.50', '1.500', '1.5000', '1.7', '1.70', '1.7000', '1.75', '10', '10.0', '1002.0', '11', '11.0', '11.00', '1120.0', '12', '12.0', '13', '13.0', '14', 

In [ ]:
realty_data1['stories'].replace(['1-2 Stories','Ground Level, Two','Two, Multi/Split','One, Two, Multi/Split','One and One Half','One, Two','Two Stories','Bungalow','Bi-Level','Ground Level, Split Level, Two','Duplex','2.0','Sixplex','Two'], 2, inplace=True)
realty_data1['stories'].replace(['1-2 Stories, 3-4 Stories','Two Story or More','One and One Half, Three Or More','Two, Three Or More','Tri-Level','Multi Level, Three or More'], 3, inplace=True)
realty_data1['stories'].replace(['3 - 5 Stories','3-4 Stories','Three Or More'], 4, inplace=True)
realty_data1['stories'].replace(['A Frame','Apartments','One','One Level','Manufactured Doublewide','One Story','1.0'], 1, inplace=True)
realty_data1['stories'].replace(['Acreage','Lot','Unimproved Commercial'], 0, inplace=True)

In [ ]:
realty_data1['stories'] = realty_data1['stories'].apply(lambda x: int(re.findall('\d+', str(x))[0]) if re.findall('\d+', str(x)) else None)

In [ ]:
missing_values_stories = realty_data1['stories'].isna().sum()



print(f"Количество пропущенных значений: {missing_values_stories}\n"
f"Количество уникальных значений: {realty_data1['stories'].nunique()}\n"
f"Уникальные значения: {list(realty_data1['stories'].sort_values().unique()[:400])}")

Количество пропущенных значений: 152006
Количество уникальных значений: 77
Уникальные значения: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 54.0, 55.0, 56.0, 57.0, 58.0, 60.0, 62.0, 63.0, 64.0, 65.0, 66.0, 68.0, 70.0, 75.0, 77.0, 78.0, 80.0, 82.0, 90.0, 91.0, 95.0, 96.0, 1002.0, 1120.0, nan]


In [ ]:
realty_data1[realty_data1.stories.isna()].Type.value_counts()

Type
single_family    47035
other            36774
land             28836
condos           25058
multi_family      5334
townhouse         3936
mobile_home       1514
ranch             1420
contemporary      1221
apartment          834
historical          44
Name: count, dtype: int64

In [ ]:
total_count = len(realty_data1)
missing_values_stories_percent = (missing_values_stories / total_count) * 100
print(missing_values_stories_percent)

40.34879091126271


Большое количество пропущенных значений даже в типах недвижимости,где есть дома. Можно постараться заполнить эти значения, но првильность заполнения будет под вопросом.

In [ ]:
realty_data1 = realty_data1.drop('stories', axis=1)

*Целевой признак*   **target**   *цена объекта недвижимости*

In [ ]:
missing_values_target  = realty_data1['target'].isna().sum()



print(f"Количество пропущенных значений: {missing_values_target}\n"
f"Количество уникальных значений: {realty_data1['target'].nunique()}\n"
f"Уникальные значения: {list(realty_data1['target'].sort_values().unique()[:40000])}")

Количество пропущенных значений: 2475
Количество уникальных значений: 43927
Уникальные значения: ['$1', '$1,000', '$1,000,000', '$1,000,000+', '$1,000,036', '$1,000,050', '$1,000,100', '$1,000,898', '$1,000/mo', '$1,001,713', '$1,001,990+', '$1,001,995', '$1,001,995+', '$1,002,500', '$1,002,562', '$1,002,695', '$1,002,990+', '$1,004,047', '$1,004,500', '$1,004,993', '$1,005,000', '$1,005,000+', '$1,005,187', '$1,005,803', '$1,006,033', '$1,006,118', '$1,006,302', '$1,006,419', '$1,006,500', '$1,006,635+', '$1,006,990', '$1,006,990+', '$1,007,000', '$1,007,112', '$1,007,530', '$1,008,000', '$1,008,098', '$1,008,375', '$1,008,800', '$1,009,000', '$1,009,034', '$1,009,995', '$1,010,000', '$1,010,214', '$1,010,990+', '$1,011,000', '$1,011,097', '$1,011,346', '$1,012,600', '$1,013,000', '$1,013,239', '$1,013,888', '$1,013,990', '$1,013,990+', '$1,014,311', '$1,014,900', '$1,014,925', '$1,014,995', '$1,014,995+', '$1,015,000', '$1,015,000+', '$1,015,554', '$1,015,666', '$1,015,722', '$1,016,

In [ ]:
# Удалим строки с пропусками так как мы не можем потом использовать
realty_data1 = realty_data1.dropna(subset=['target'])

В строках есть данные о цене за месяц (с частью '/mo')

In [ ]:
realty_data1[realty_data1.target.str.contains('/mo',regex=True)].head()

,status,street,baths,fireplace,city,sqft,zipcode,state,target,Type,pool,Year built,Heating,Cooling,Parking,school_rating _mean,school_dist_min
547,For rent,4323 N Central Park Ave,3.0,True,Chicago,3300.0,60618,IL,"$5,500/mo",single_family,False,1913,True,False,True,2.3,0.13
609,For rent,220 Boylston St #1412,2.0,True,Boston,1673.0,2116,MA,"$10,500/mo",multi_family,False,1985,False,False,False,-1.0,-1.00
2075,For rent,2830 NE 56th Ct,4.0,False,Fort Lauderdale,2400.0,33308,FL,"$6,390/mo",single_family,True,1965,False,False,False,4.0,1.19
3025,For rent,411 Kline Aly,2.0,False,Clarksville,1280.0,37040,TN,"$1,200/mo",multi_family,False,2014,True,False,False,8.0,0.68
3645,For rent,240 E Illinois St #2011,2.0,False,Chicago,1473.0,60611,IL,"$3,600/mo",multi_family,True,2003,False,False,True,5.5,0.72


Так как мы прогнозируем цену на продажу квартиры,а не на аренду ,то удалим их.

In [ ]:
realty_data1 = realty_data1[~realty_data1['target'].str.contains('/mo', regex=True)]

In [ ]:
# Замена символов в столбце 'target'
realty_data1['target'] = realty_data1['target'].str.replace(r'\$|\+|,|', '', regex=True)

In [ ]:
realty_data1.target = realty_data1.target.astype(int)

In [ ]:
display(realty_data1[realty_data1['target'] == realty_data1['target']. min ()].head(10))
display(realty_data1[realty_data1['target'] == realty_data1['target']. max ()].head())

,status,street,baths,fireplace,city,sqft,zipcode,state,target,Type,pool,Year built,Heating,Cooling,Parking,school_rating _mean,school_dist_min
15533,Active,1230 Old Dickerson Rd,2.0,False,Goodlettsville,1596.0,37072,TN,1,single_family,False,1940,True,False,True,2.3,0.20
25334,other,3205 N High School Rd,1.0,True,Indianapolis,1586.0,46224,IN,1,single_family,False,1957,True,True,True,1.5,0.20
84943,Foreclosed,7804 Hamilton Ave,2.0,True,Cincinnati,2536.0,45231,OH,1,land,False,1865,False,True,True,2.0,0.50
148218,Active,Cummings St,2.0,False,Huntley,0.0,60142,IL,1,land,False,no date,False,False,False,-1.0,-1.00
205313,Active,Route 47 Hwy,2.0,False,Huntley,0.0,60142,IL,1,land,False,no date,False,False,False,7.8,0.90
252231,Foreclosed,8136 Bradyville Pike,2.0,False,Murfreesboro,0.0,37127,TN,1,land,False,2005,False,False,False,6.3,3.15
252558,Active,114 S Washington St,2.0,True,Circleville,1771.0,43113,OH,1,single_family,False,1920,False,True,False,4.7,1.00
256120,other,9115 Sea Oats Dr,3.0,True,Indianapolis,1780.0,46250,IN,1,condos,False,1989,True,True,True,3.7,1.90
270972,Active,Middleground Rd SW,2.0,False,Pataskala,0.0,43062,OH,1,land,False,no date,False,False,False,6.8,1.80
277489,Foreclosed,2902 W McMicken Ave,3.0,True,Cincinnati,3226.0,45225,OH,1,multi_family,False,1870,True,True,True,2.5,0.59


,status,street,baths,fireplace,city,sqft,zipcode,state,target,Type,pool,Year built,Heating,Cooling,Parking,school_rating _mean,school_dist_min
132406,Active,875 Nimes Rd,18.0,True,Los Angeles,0.0,90077,CA,195000000,single_family,True,1935,True,True,True,2.7,1.3


In [ ]:
#  количество строк
count = len(realty_data1[realty_data1['target'] < 500])
print(f"Количество строк: {count}")


Количество строк: 25


Удаляем строки,где стоимость недвижимости меньше 500 ,  так как в среднем по рынку даже небольшой участок земли(без дома)стоит около 500-1000

In [ ]:
realty_data1 = realty_data1.drop(realty_data1[realty_data1['target'] < 500].index)

In [ ]:
missing_values_target  = realty_data1['target'].isna().sum()
print(f"Количество пропущенных значений: {missing_values_target}")

Количество пропущенных значений: 0


In [ ]:
# Проверим дубликаты
realty_data1.duplicated().sum()

306

In [ ]:
# Удалим дубликаты
realty_data1 = realty_data1.drop_duplicates(ignore_index=True)

In [ ]:
realty_data1.head()

,status,street,baths,fireplace,city,sqft,zipcode,state,target,Type,pool,Year built,Heating,Cooling,Parking,school_rating _mean,school_dist_min
0,Active,240 Heather Ln,3.0,True,Southern Pines,2900.0,28387,NC,418000,single_family,False,2019,True,False,False,5.2,2.70
1,Active,12911 E Heroy Ave,3.0,False,Spokane Valley,1947.0,99216,WA,310000,single_family,False,2019,False,False,False,4.0,1.01
2,Active,2005 Westridge Rd,2.0,True,Los Angeles,3000.0,90049,CA,2895000,single_family,True,1961,True,True,True,6.7,1.19
3,Active,4311 Livingston Ave,8.0,True,Dallas,6457.0,75205,TX,2395000,single_family,False,2006,True,True,True,5.6,0.10
4,Active,1524 Kiscoe St,2.0,False,Palm Bay,0.0,32908,FL,5000,land,False,no date,False,False,False,4.7,3.03


Посмотрим на строки,которые содержат большое количество пропусков

In [ ]:
uninform_lines = realty_data1[(realty_data1['sqft']==0) &
                              (realty_data1['Year built'] == 'no date') &
                              (realty_data1['baths']==0)]
display(uninform_lines.head())

num_uninform_lines = len(uninform_lines)

print(f"{num_uninform_lines} строк с пропусками ")

,status,street,baths,fireplace,city,sqft,zipcode,state,target,Type,pool,Year built,Heating,Cooling,Parking,school_rating _mean,school_dist_min
287,Pending,Goodhope,0.0,False,Houston,0.0,77021,TX,39900,land,False,no date,False,False,False,2.3,0.2
355,Active,1449 Tern Ct,0.0,False,Poinciana,0.0,34759,FL,23000,land,False,no date,False,False,False,2.2,1.2
577,Active,1211 Molona St,0.0,False,Reunion,0.0,34747,FL,42800,land,False,no date,False,False,False,5.4,2.1
616,Active,4719 Rue St,0.0,False,Houston,0.0,77033,TX,17500,land,False,no date,False,False,False,2.3,0.5
843,Active,120 Smallwood Rd,0.0,False,Rotonda West,0.0,33947,FL,9700,land,False,no date,False,False,False,5.7,2.3


3802 строк с пропусками 


Удаление строк,где тип недвижимости не земля(земельный участок),нет ванн и год постройки неизвестен.

In [ ]:
realty_data1 = realty_data1[
    ~((realty_data1['sqft'] == 0) &
      (realty_data1['Year built'] == 'no date') &
      (realty_data1['baths'] == 0))
]

In [ ]:
display(realty_data1.sample(5))
display(realty_data1.info())

,status,street,baths,fireplace,city,sqft,zipcode,state,target,Type,pool,Year built,Heating,Cooling,Parking,school_rating _mean,school_dist_min
313218,Active,1143 Fedora St,5.0,False,Los Angeles,3400.0,90006,CA,1119000,multi_family,False,1925,True,False,False,2.0,0.10
256477,Active,6711 W 38th Ave,2.0,False,Kennewick,1354.0,99338,WA,279000,single_family,True,2019,True,True,False,4.0,0.54
13181,Active,5081 Pine Mountain Ave,3.0,False,Las Vegas,1367.0,89139,NV,275000,single_family,False,2004,True,True,True,5.0,1.30
142156,Active,31 Richardson Dr,2.0,False,Palm Coast,0.0,32164,FL,24500,land,False,no date,False,False,False,4.7,0.23
183283,Active,1819 E Outer Dr,2.0,True,Detroit,1770.0,48234,MI,60000,single_family,False,1946,True,False,True,3.0,1.79


<class 'pandas.core.frame.DataFrame'>
Index: 369724 entries, 0 to 373525
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   status               369724 non-null  object 
 1   street               369724 non-null  object 
 2   baths                369724 non-null  float64
 3   fireplace            369724 non-null  bool   
 4   city                 369724 non-null  object 
 5   sqft                 369724 non-null  float64
 6   zipcode              369724 non-null  object 
 7   state                369724 non-null  object 
 8   target               369724 non-null  int32  
 9   Type                 369724 non-null  object 
 10  pool                 369724 non-null  bool   
 11  Year built           369724 non-null  object 
 12  Heating              369724 non-null  bool   
 13  Cooling              369724 non-null  bool   
 14  Parking              369724 non-null  bool   
 15  school_rating _mean  3

None

Сохраним,чтобы не потерять при сбоях

In [ ]:
realty_data1.to_csv("realty_data_full.csv", index=False)